<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman;">
    TIME SERIES DATA PROCESSING - Main Notebook
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Each part of the following script was used to proccess the raw data for the Availability Factor Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    1. Notebook Set Up
</div>

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Importing needed libraries
</div>

In [141]:
import pandas as pd
import re
import csv
import os
import requests
from urllib.parse import urlparse, parse_qs

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    2. Dispa-SET_Unleash Folder Path
</div>

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory. 
<br>
    If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [142]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
3. European Standard Time
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Each of the Europeand Countries Modeled in Dispa-SET are in a particular european standar time zone. So in order to get the appropiated time series data frame, the correspondign european time zone is going to be identificated for each country.
<br>
Additionally all the time series is also going to be downloaded under the UTC <em>(The World’s Time Standard)</em>.
<br>
This is going to be done for each country as well.
    <br>
All this features are going to be saved in a csv file called Standar_Time_Data.csv where all additional characters to the download process is going to be written
</div>

In [143]:
# Define lists of countries and standard times
countries = [
    "Austria", "Belgium", "Bulgaria", "Switzerland", "Cyprus", "Czech Republic",
    "Germany", "Denmark", "Estonia", "Greece", "Spain", "Finland", "France",
    "Croatia", "Hungary", "Ireland", "Italy", "Lithuania", "Luxembourg", "Latvia",
    "Malta", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Sweden",
    "Slovenia", "Slovakia", "United Kingdom"
]

standard_times = [
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Greenwich Mean Time (GMT) or Irish Standard Time (IST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Western European Time (WET) or Western European Summer Time (WEST) when daylight saving time is observed",
    "Eastern European Time (EET) or Eastern European Summer Time (EEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Central European Time (CET) or Central European Summer Time (CEST) when daylight saving time is observed",
    "Greenwich Mean Time (GMT) or British Summer Time (BST) when daylight saving time is observed"
]

# Create DataFrame
df = pd.DataFrame({'Country': countries, 'Full_Standard_Time': standard_times})

# Add new empty column
df['Standard_Time'] = ''

df

,Country,Full_Standard_Time,Standard_Time
0,Austria,Central European Time (CET) or Central Europea...,
1,Belgium,Central European Time (CET) or Central Europea...,
2,Bulgaria,Eastern European Time (EET) or Eastern Europea...,
3,Switzerland,Central European Time (CET) or Central Europea...,
4,Cyprus,Eastern European Time (EET) or Eastern Europea...,
5,Czech Republic,Central European Time (CET) or Central Europea...,
6,Germany,Central European Time (CET) or Central Europea...,
7,Denmark,Central European Time (CET) or Central Europea...,
8,Estonia,Eastern European Time (EET) or Eastern Europea...,
9,Greece,Eastern European Time (EET) or Eastern Europea...,


In [144]:
# Function to extract words in () and return as a string
def extract_words_in_brackets(text):
    matches = re.findall(r'\((.*?)\)', text)
    return ' '.join(matches)

# Apply the function to the second column and assign the result to the third column
df['Standard_Time'] = df['Full_Standard_Time'].apply(extract_words_in_brackets)

df

,Country,Full_Standard_Time,Standard_Time
0,Austria,Central European Time (CET) or Central Europea...,CET CEST
1,Belgium,Central European Time (CET) or Central Europea...,CET CEST
2,Bulgaria,Eastern European Time (EET) or Eastern Europea...,EET EEST
3,Switzerland,Central European Time (CET) or Central Europea...,CET CEST
4,Cyprus,Eastern European Time (EET) or Eastern Europea...,EET EEST
5,Czech Republic,Central European Time (CET) or Central Europea...,CET CEST
6,Germany,Central European Time (CET) or Central Europea...,CET CEST
7,Denmark,Central European Time (CET) or Central Europea...,CET CEST
8,Estonia,Eastern European Time (EET) or Eastern Europea...,EET EEST
9,Greece,Eastern European Time (EET) or Eastern Europea...,EET EEST


In [145]:
# Additional string to be appended
additional_path = "/RawData/AvailabiltyFactors/"

# Construct the power_plants_raw_data_folder_path variable
standard_time_data_folder_path = dispaSET_unleash_folder_path + additional_path
print("standard_time_data_folder_path:", standard_time_data_folder_path)

standard_time_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/


In [146]:
standard_time_data_file_name = 'Standard_Time_Data.csv'

# Save DataFrame to a CSV file
standard_time_data_file_path = standard_time_data_folder_path + standard_time_data_file_name
df.to_csv(standard_time_data_file_path, index=False)

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [147]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"standard_time_data_file_name:                              {standard_time_data_file_name}")
print (f"standard_time_data_file_path:                              {standard_time_data_file_path}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
standard_time_data_file_name:                              Standard_Time_Data.csv
standard_time_data_file_path:                              /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
3. Main Data Source
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
All the data to be processed is going to be downloaded from one main source:
<br>
<em><strong>ENTSOE Transparency Platform:</strong></em> Which the main url link is the following
<div style="text-align: left; margin-left: 3.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
https://transparency.entsoe.eu/dashboard/show
</div>

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;">
3.1. Url Dowload Link Base
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
All the downloaded data use a base url structure as download link.
<br>
The URL base is lightly modified to get the correponding for each country.
    <br>
Each URL_country_base link is added to the csv file standard_time_data_file_path
    <br>
The ENTSOE web portal do not provide the corresponding information for Malta. This country is going to be avoided in the tables of the next process. However, other sources are going to be looked for and their corresponding dowload and formating scrips are going to be developed later.
</div>

In [148]:
# List of URLs
urls = [
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YAT-APG------L!CTY|10YAT-APG------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTY|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A65H!CTY|10Y1001A1001A65H&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YGR-HTSO-----Y!CTY|10YGR-HTSO-----Y&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YES-REE------0!CTY|10YES-REE------0&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFI-1--------U!CTY|10YFI-1--------U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFR-RTE------C!CTY|10YFR-RTE------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHR-HEP------M!CTY|10YHR-HEP------M&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHU-MAVIR----U!CTY|10YHU-MAVIR----U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIE-1001A00010!CTY|10YIE-1001A00010&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIT-GRTN-----B!CTY|10YIT-GRTN-----B&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLT-1001A0008Q!CTY|10YLT-1001A0008Q&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLU-CEGEDEL-NQ!CTY|10YLU-CEGEDEL-NQ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLV-1001A00074!CTY|10YLV-1001A00074&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    " ",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNL----------L!CTY|10YNL----------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNO-0--------C!CTY|10YNO-0--------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YRO-TEL------P!CTY|10YRO-TEL------P&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSE-1--------K!CTY|10YSE-1--------K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSI-ELES-----O!CTY|10YSI-ELES-----O&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSK-SEPS-----K!CTY|10YSK-SEPS-----K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A92E!CTY|10Y1001A1001A92E&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC"
]

# Read the CSV file
df = pd.read_csv(standard_time_data_file_path)

# Add a new column "Url_Country_Base" and populate it with the URLs
df["Url_Country_Base"] = urls

# Write the DataFrame back to the CSV file
df.to_csv(standard_time_data_file_path, index=False)

print("URLs have been added to the new column 'Url_Country_Base' in the CSV file:", standard_time_data_file_path)

URLs have been added to the new column 'Url_Country_Base' in the CSV file: /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv


<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
The country zone for each country is stracted in a new column.
<br>
This data is needed to perform the next download links recursively.
</div>

In [150]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv(standard_time_data_file_path)

# Function to extract country code from the URL
def extract_country_code(url):
    start_index = url.find("values=CTY|") + len("values=CTY|")
    end_index = url.find("!CTY|")
    if start_index >= len("values=CTY|") and end_index > start_index:
        return url[start_index:end_index]
    else:
        return None

# Apply the function to extract country code for each row
data['Country_Code'] = data['Url_Country_Base'].apply(extract_country_code)

# Write the DataFrame back to the CSV file with the new column included
data.to_csv(standard_time_data_file_path, index=False)

In [151]:
data

,Country,Full_Standard_Time,Standard_Time,Url_Country_Base,Country_Code
0,Austria,Central European Time (CET) or Central Europea...,CET CEST,https://transparency.entsoe.eu/generation/r2/a...,10YAT-APG------L
1,Belgium,Central European Time (CET) or Central Europea...,CET CEST,https://transparency.entsoe.eu/generation/r2/a...,10YBE----------2
2,Bulgaria,Eastern European Time (EET) or Eastern Europea...,EET EEST,https://transparency.entsoe.eu/generation/r2/a...,10YCA-BULGARIA-R
3,Switzerland,Central European Time (CET) or Central Europea...,CET CEST,https://transparency.entsoe.eu/generation/r2/a...,10YCH-SWISSGRIDZ
4,Cyprus,Eastern European Time (EET) or Eastern Europea...,EET EEST,https://transparency.entsoe.eu/generation/r2/a...,10YCY-1001A0003J
5,Czech Republic,Central European Time (CET) or Central Europea...,CET CEST,https://transparency.entsoe.eu/generation/r2/a...,10YCZ-CEPS-----N
6,Germany,Central European Time (CET) or Central Europea...,CET CEST,https://transparency.entsoe.eu/generation/r2/a...,10Y1001A1001A83F
7,Denmark,Central European Time (CET) or Central Europea...,CET CEST,https://transparency.entsoe.eu/generation/r2/a...,10Y1001A1001A65H
8,Estonia,Eastern European Time (EET) or Eastern Europea...,EET EEST,https://transparency.entsoe.eu/generation/r2/a...,10Y1001A1001A83F
9,Greece,Eastern European Time (EET) or Eastern Europea...,EET EEST,https://transparency.entsoe.eu/generation/r2/a...,10YGR-HTSO-----Y


In [140]:
# Function to extract country codes from URLs
def extract_country_code(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    area_values = query_params.get('area.values')
    if area_values:
        return area_values[0].split('|')[1][:2]
    return None

# Read the CSV file
df = pd.read_csv(standard_time_data_file_path)

# Extract country codes from URLs in the "Url_Country_Base" column
df['Country_Zone_Code'] = df['Url_Country_Base'].apply(extract_country_code)

# Write the DataFrame back to the CSV file
df.to_csv(standard_time_data_file_path, index=False)

print("Country zone codes have been added to the new column 'Country_Zone_Code' in the CSV file:", standard_time_data_file_path)

Country zone codes have been added to the new column 'Country_Zone_Code' in the CSV file: /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv


In [3]:
url = "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTA&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=02.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=02.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YAT-APG------L!CTA|10YAT-APG------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)"

# Find the index of "values=CTY|"
start_index = url.find("values=CTY|")

# Extract the substring after "values=CTY|"
substring = url[start_index + len("values=CTY|"):start_index + len("values=CTY|") + 36]

print(substring)

10YAT-APG------L!CTA|10YAT-APG------


In [5]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the webpage containing the table
url = 'https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTA&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=11.02.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTA|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)'

# Send a GET request to the webpage
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table element
table = soup.find('table')

# Extract data from the table
data = []
for row in table.find_all('tr'):
    row_data = []
    for cell in row.find_all('td'):
        row_data.append(cell.text.strip())
    if row_data:  # Ensures we don't add empty rows
        data.append(row_data)

# Write the data to a CSV file
with open('table_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

In [6]:
for row in data:
    print(row)

['00:00 - 01:00', '268', '273', '281', '', '', '', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', '2953', '3573', '1995', '', '', '', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', '0', '0', '0', '', '', '', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', '', '', '116', '390', '105', '', '36', '32', '30', '', '', '', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', '3951', '3949', '3952', '', '', '', '767', '829', '905', '', '', '', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', '0', '0', '0', '', '', '', '221', '227', '232', '', '', '', '464', '53', '590', '', '', '', '768', '141', '844', '', '', '']
['01:00 - 02:00', '270', '271', '283', '', '', '', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', '2623', '3415', '2036', '', '', '', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', 'n/e', '0', '0', '0', '', '', '', 'n/e', 'n/e

In [7]:
import pandas as pd

# Convert the data list into a pandas DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

              0    1    2    3   4   5   6    7    8    9    ...  111 112 113  \
0   00:00 - 01:00  268  273  281              n/e  n/e  n/e  ...  590           
1   01:00 - 02:00  270  271  283              n/e  n/e  n/e  ...  641           
2   02:00 - 03:00  269  274  282              n/e  n/e  n/e  ...  750           
3   03:00 - 04:00  269  272  285              n/e  n/e  n/e  ...  713           
4   04:00 - 05:00  267  274  279              n/e  n/e  n/e  ...  529           
5   05:00 - 06:00  264  272  278              n/e  n/e  n/e  ...  542           
6   06:00 - 07:00  271  272  274              n/e  n/e  n/e  ...  490           
7   07:00 - 08:00  270  275  276              n/e  n/e  n/e  ...  452           
8   08:00 - 09:00  273  273  273              n/e  n/e  n/e  ...  431           
9   09:00 - 10:00  271  274  273              n/e  n/e  n/e  ...  499           
10  10:00 - 11:00  268  277  277              n/e  n/e  n/e  ...  398           
11  11:00 - 12:00  273  277 

In [2]:
base_url = 'https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTA&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTA|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)'

In [ ]:
from datetime import datetime, timedelta

# Define the start date
start_date = datetime(2023, 1, 1)

# Define the end date
end_date = datetime(2023, 12, 31)

# File path to save the dates
file_path = "2023_dates.txt"  # Change this to your desired file path

# Open the file in write mode
with open(file_path, 'w') as file:
    # Generate all the days between start and end date
    current_date = start_date
    while current_date <= end_date:
        # Format the date in the desired format
        formatted_date = current_date.strftime('%d.%m.%Y+00:00')
        
        # Write the formatted date to the file
        file.write(formatted_date + '\n')
        
        # Move to the next day
        current_date += timedelta(days=1)
